# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

weather_df = pd.read_csv("cities.csv")
weather_df

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Date
0,0,ilulissat,GL,69.2167,-51.1000,100.0,93.0,30.22,0.00,1.664596e+09
1,1,mahebourg,MU,-20.4081,57.7000,75.0,68.0,70.09,12.66,1.664596e+09
2,2,kaitangata,NZ,-46.2817,169.8464,0.0,59.0,48.33,5.50,1.664596e+09
3,3,los llanos de aridane,ES,28.6585,-17.9182,100.0,84.0,65.95,10.36,1.664596e+09
4,4,kavieng,PG,-2.5744,150.7967,73.0,73.0,82.98,13.53,1.664596e+09
...,...,...,...,...,...,...,...,...,...,...
319,319,kodiak,US,57.7900,-152.4072,0.0,71.0,47.71,0.00,1.664596e+09
320,320,saint anthony,US,45.0205,-93.2180,20.0,59.0,64.18,8.05,1.664596e+09
321,321,kachiry,KZ,53.0675,76.1036,14.0,54.0,37.35,7.61,1.664596e+09
322,322,taoudenni,ML,22.6783,-3.9836,1.0,20.0,79.47,8.81,1.664596e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)



In [6]:
# Takeout any NaN values
weather_df = weather_df.dropna()
# Convert Humidity to float and store
humidity = weather_df["Humidity (%)"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
cleaned_weather_df = weather_df

# A max temperature lower than 85 degrees but higher than 70.
cleaned_weather_df = cleaned_weather_df.loc[(cleaned_weather_df["Max Temp (F)"] < 85) & (cleaned_weather_df["Max Temp (F)"] > 70)]

# Wind speed less than 5 mph.
cleaned_weather_df = cleaned_weather_df.loc[cleaned_weather_df["Wind Speed (mph)"] < 5]

# Zero cloudiness.
cleaned_weather_df = cleaned_weather_df.loc[cleaned_weather_df["Cloudiness (%)"] == 0]

# Drop any rows with null values
cleaned_weather_df = cleaned_weather_df.dropna()

cleaned_weather_df

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Date
59,59,gat,IL,31.6100,34.7642,0.0,84.0,72.28,1.83,1.664596e+09
69,69,riyadh,SA,24.6877,46.7219,0.0,15.0,71.74,3.47,1.664596e+09
112,112,antakya,TR,36.2066,36.1572,0.0,58.0,73.22,3.67,1.664596e+09
136,136,sitio novo do tocantins,BR,-5.5183,-47.4611,0.0,69.0,82.33,4.61,1.664596e+09
141,141,sweetwater,US,25.7634,-80.3731,0.0,88.0,77.02,0.00,1.664596e+09
154,154,wajir,KE,1.7471,40.0573,0.0,68.0,71.56,0.00,1.664596e+09
171,171,cayenne,GF,4.9333,-52.3333,0.0,94.0,75.24,0.00,1.664596e+09
228,228,laguna,US,38.4210,-121.4238,0.0,45.0,79.09,1.99,1.664596e+09
258,258,ambovombe,MG,-25.1667,46.0833,0.0,62.0,70.27,4.97,1.664596e+09
316,316,karnal,IN,29.6833,76.9833,0.0,67.0,82.71,2.10,1.664596e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = cleaned_weather_df

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 59: gat.
Missing field/result... skipping.
------------
Retrieving Results for Index 69: riyadh.
Missing field/result... skipping.
------------
Retrieving Results for Index 112: antakya.
Missing field/result... skipping.
------------
Retrieving Results for Index 136: sitio novo do tocantins.
Missing field/result... skipping.
------------
Retrieving Results for Index 141: sweetwater.
Missing field/result... skipping.
------------
Retrieving Results for Index 154: wajir.
Missing field/result... skipping.
------------
Retrieving Results for Index 171: cayenne.
Missing field/result... skipping.
------------
Retrieving Results for Index 228: laguna.
Missing field/result... skipping.
------------
Retrieving Results for Index 258: ambovombe.
Missing field/result... skipping.
------------
Retrieving Results for Index 316: karnal.
Missing field/result... skipping.
------------


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
